In [ ]:
# If we don't have the data, download it
!wget https://dmlab.cs.gsu.edu/solar/data/data-comp-2020/train_partition1_data.json
!mkdir data
!mv train_partition1_data.json data/

In [1]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import counter, subSample
from torch.utils.data import DataLoader
from normalizer import subSample, getDataFromJSON

device = 'cuda' if torch.cuda.is_available() else 'cpu' # Change this to cuda for GPU enabled computers
device

/home/robjohnson/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


'cpu'

In [2]:
train1, labels1 = subSample("data/train_partition1_data.json", device=device)

Now reading data/train_partition1_data.json
Now loading event 1/785
Now loading event 101/785
Now loading event 201/785
Now loading event 301/785
Now loading event 401/785
Now loading event 501/785
Now loading event 601/785
Now loading event 701/785
785 lines loaded.


In [3]:
for X in [train1]: # This line edited for easier loading
    for i in range(X.shape[1]): # the number of types of measurements
        min_x = torch.min(X[:,i,:])
        X[:,i,:] -= min_x
        X[:,i,:] /= torch.max(X[:,i,:]).item()

In [4]:
# Define the network. Make sure to end with nn.Softmax activation
import torch.nn as nn
# from skorch import NeuralNet

class logRegWithHidden(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, num_classes=5, drop1=.5, input_size=1980):
        super().__init__() 
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layerout = nn.Linear(hidden_size2, num_classes)
        #Define a RELU Activation unit
        self.relu = nn.ReLU()  
        self.smax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=drop1)

    def forward(self, x):
        #Forward Propagate through the layers as defined above
        y = self.drop(x.reshape(-1, 1980))
        y = self.drop(self.relu(self.layer1(y)))
        y = self.relu(self.layer2(y))
        y = self.layerout(y)
        y = self.smax(y)
        return y



In [15]:
def train(modelModule, inputs, labels, weight, valSets, valLabels, valweight, *modelArgs, lr=0.0001, **modelKwargs):
    
    # modify these variables to control how much is printed
    pmodel = False
    pstateDict = False
    pindvLoss = False
    pmodelDict = False
    pvalOut = False
    pbl = False
    checkClone = True
    
    
    #TODO: does this work?
    model = modelModule(*modelArgs, **modelKwargs)
    if checkClone: 
        PATH = './badModel.pth'
        torch.save(model.state_dict(), PATH)
    if pmodel: print(model)
    
    if weight is not None: weight = torch.Tensor(weight)
    lfc = nn.CrossEntropyLoss(weight=weight)
#     valLabels = torch.tensor(valLabels, dtype=torch.int)
    #ideas
    # 1-(weight/np.sum(weight))
    # .2/weight - this one normalizes so that each class is responsible for 20% of the loss
    # 1/weight - this is a bit naive, but the classes with fewer items are weighted more.
    # 1/(weight+1) - makes sure we don't have any pesky zeroes
    # np.sum(weight)/weight if your learning rate is too low.
    
    # Hyperparameters
    batch = 256
    epochs = 40 # artificially low for debugging
    
    # Start a dataloader object
    data = list(zip(inputs,labels))
    val = list(zip(valSets,valLabels))
    loader = DataLoader(data, batch_size = batch, num_workers=4)
    valLoader = DataLoader(val, batch_size = int(len(val)/4), num_workers=4)
    opt = torch.optim.SGD(model.parameters(), lr=lr)
    if pstateDict: print(opt.state_dict())

    for epoch in range(epochs):
        model.train()
        batch_loss = []
        if pbl: print(batch_loss)
        for (xtrain,ytrain) in loader:
            opt.zero_grad()
            if pstateDict: print('Opt dict post zero grad:', '================',opt.state_dict(), sep='\n')
            output = model(xtrain)
            loss = lfc(output,ytrain)
            if pindvLoss: print(loss)
            loss.backward()
            if pstateDict: print('Opt dict post backward:', '================',opt.state_dict(), sep='\n')
            if pmodelDict: print(model.state_dict())
            opt.step()
            if pstateDict: print('Opt dict post step:', '================',opt.state_dict(), sep='\n')
            if pstateDict or pmodelDict: print('\n\n\n\n\n')
            batch_loss.append(loss.item())
        print(f'The training loss for epoch {epoch+1}/{epochs} was {np.mean(batch_loss)}')
        if pbl: print(batch_loss)
        
        model.eval()
        balanced = [[],[],[],[],[]]
        batchLoss = []
        unbalanced = []
        
        for (xval,yval) in valLoader:
            output = model(xval)
            loss = lfc(output,yval)
            batchLoss.append(loss.item())
            if pvalOut : print(output)
            corrects = yval.clone().detach() == torch.argmax(output,1)
            if pvalOut: print(yval.clone().detach(), torch.argmax(output,1))
            if pvalOut: print(corrects.detach())
            if pvalOut: print('===========================\n\n\n')
            unbalanced.append(np.mean([1 if correct else 0 for correct in corrects.detach()]))
            
            for i, ans in enumerate(yval):
                balanced[ans].append(corrects[i])
        
        balanced = [np.mean(i) for i in balanced]
        balancedAccuracy = np.mean(balanced)
        
#         print(f'The total balanced accuracy for validation was {balancedAccuracy}')
        print(f'The values of unbalanced are {unbalanced}')
        print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')
        print(f'The unbalanced validation accuracy is {np.mean(unbalanced)}')
        print(f'The accuracy for each is {balanced}')       
        
        
        if checkClone:
            fives = modelModule(*modelArgs, **modelKwargs)
            fives.load_state_dict(torch.load(PATH))
            s=2
            feature_extraction1 = [child for child in model.children()]
            print(feature_extraction1[s])
            feature_extraction2 = [child for child in fives.children()]
            print(feature_extraction2[s])
            print(torch.max(feature_extraction1[s].weight - feature_extraction2[s].weight).detach())
            print(torch.min(feature_extraction1[s].weight - feature_extraction2[s].weight).detach())
    if pstateDict: print(opt.state_dict())


    return model


In [20]:
# [weights1[i] + weights2[i] for i in range(5)]
trainArgs = [logRegWithHidden, train1, labels1, None, train1, labels1, None]
modelArgs = [4096*4, 2048*4]
lr = 0.01
%time newModel = train(*trainArgs, *modelArgs, lr = lr, drop1=0)
lr

The training loss for epoch 1/40 was 1.6070178747177124
The values of unbalanced are [0.22448979591836735, 0.24489795918367346, 0.2653061224489796, 0.32142857142857145, 0.0]
The validation loss was :   1/40 was 1.607279396057129
The unbalanced validation accuracy is 0.21122448979591835
The accuracy for each is [0.0, 0.7961783439490446, 0.5222929936305732, 0.0, 0.0]
Linear(in_features=8192, out_features=5, bias=True)
Linear(in_features=8192, out_features=5, bias=True)
tensor(0.0002)
tensor(-0.0002)
The training loss for epoch 2/40 was 1.6033762097358704
The values of unbalanced are [0.19387755102040816, 0.22959183673469388, 0.21428571428571427, 0.25, 0.0]
The validation loss was :   2/40 was 1.604910397529602
The unbalanced validation accuracy is 0.17755102040816326
The accuracy for each is [0.0, 1.0, 0.10828025477707007, 0.0, 0.0]
Linear(in_features=8192, out_features=5, bias=True)
Linear(in_features=8192, out_features=5, bias=True)
tensor(0.0004)
tensor(-0.0004)
The training loss for 

Linear(in_features=8192, out_features=5, bias=True)
Linear(in_features=8192, out_features=5, bias=True)
tensor(0.0026)
tensor(-0.0027)
The training loss for epoch 18/40 was 1.562122493982315
The values of unbalanced are [0.413265306122449, 0.42346938775510207, 0.40816326530612246, 0.4030612244897959, 1.0]
The validation loss was :   18/40 was 1.554177212715149
The unbalanced validation accuracy is 0.5295918367346939
The accuracy for each is [0.5222929936305732, 0.8662420382165605, 0.0, 0.0, 0.6751592356687898]
Linear(in_features=8192, out_features=5, bias=True)
Linear(in_features=8192, out_features=5, bias=True)
tensor(0.0028)
tensor(-0.0029)
The training loss for epoch 19/40 was 1.5595931708812714
The values of unbalanced are [0.3979591836734694, 0.4336734693877551, 0.40816326530612246, 0.40816326530612246, 1.0]
The validation loss was :   19/40 was 1.549363875389099
The unbalanced validation accuracy is 0.5295918367346939
The accuracy for each is [0.5732484076433121, 0.80254777070063

Linear(in_features=8192, out_features=5, bias=True)
Linear(in_features=8192, out_features=5, bias=True)
tensor(0.0066)
tensor(-0.0053)
The training loss for epoch 34/40 was 1.5135201513767242
The values of unbalanced are [0.44387755102040816, 0.45918367346938777, 0.413265306122449, 0.44387755102040816, 1.0]
The validation loss was :   34/40 was 1.4571537256240845
The unbalanced validation accuracy is 0.5520408163265306
The accuracy for each is [0.8662420382165605, 0.5477707006369427, 0.0, 0.0, 0.7898089171974523]
Linear(in_features=8192, out_features=5, bias=True)
Linear(in_features=8192, out_features=5, bias=True)
tensor(0.0069)
tensor(-0.0055)
The training loss for epoch 35/40 was 1.5102699995040894
The values of unbalanced are [0.44387755102040816, 0.4642857142857143, 0.41836734693877553, 0.4489795918367347, 1.0]
The validation loss was :   35/40 was 1.4512407541275025
The unbalanced validation accuracy is 0.5551020408163265
The accuracy for each is [0.8789808917197452, 0.5477707006

0.01

In [24]:
from datetime import datetime
acc = 
PATH = f'savedModels/lr{lr}acc{acc}time{datetime.now().strftime("%d_%H:%M")}.pth'
torch.save(newModel.state_dict(), PATH)
print('REMEMBER TO DELETE YOUR ACCURACY SO THE NEXT PERSON REMEMBERS TO WRITE THEIRS')